# BeyondMimic
### BeyondMimic is a versatile humanoid control framework developed by HybridRobotics that provides highly dynamic motion tracking with state-of-the-art motion quality on real-world deployment and steerable test-time control with guided diffusion-based controllers 


### The BeyondMimic pipeline is divided into two main components:

***Training Phase: whole_body_tracking/***

***Deployment Phase: motion_tracking_controller/***

# Training Phase: whole_body_tracking

### **The training phase uses Isaac Lab to train motion tracking policies from reference motion data, and WandB for policy cloud upload.**

## Preparing a Python Environment

In [ ]:


# Create an account from the WandB website :
https://wandb.ai/site/ 

# Installation of Isaac Lab v2.1.0 based on the official installation guide using conda installation method
# Isaac Sim 4.5, the required Python version is 3.10
#conda remove --name isaacLab --all
conda create -n isaacLab python=3.10 -y
conda activate isaacLab

#sanity check
which python
which pip

echo $PYTHONPATH
#In case of different Python paths or ROS environments, a conflict may occur. The simplest and most effective solution is to unset the Python path before installation.
# unset PYTHONPATH
# export PYTHONNOUSERSITE=1  to ignore ~/.local

# Upgrade pip to ensure the latest pip version is installed
# purge pip before to make sure everything is okay, the processed to the installation
python -m pip cache purge

#sudo pip install --upgrade pip
pip install --upgrade pip

# Or
python -m pip install --upgrade --ignore-installed pip setuptools wheel

# Install Isaac Sim pip packages:
pip install "isaacsim[all,extscache]==4.5.0" --extra-index-url https://pypi.nvidia.com

# Install a CUDA-enabled PyTorch build that matches your system architecture:
pip install -U torch==2.7.0 torchvision==0.22.0 --index-url https://download.pytorch.org/whl/cu128

# Verifying the Isaac Sim installation
isaacsim

# Note: The first run will prompt users to accept the Nvidia Omniverse License Agreement. To accept the EULA, reply Yes when prompted with the below message:

By installing or using Omniverse Kit, I agree to the terms of NVIDIA OMNIVERSE LICENSE AGREEMENT (EULA)
in https://docs.omniverse.nvidia.com/platform/latest/common/NVIDIA_Omniverse_License_Agreement.html

Do you accept the EULA? (Yes/No): Yes


## Installing Isaac Lab 

In [ ]:

# Cloning Isaac Lab
cd ~/RL/BeyondMimic
git clone git@github.com:isaac-sim/IsaacLab.git 

 # Installation

# these dependency are needed by robomimic which is not available on Windows
sudo apt install cmake build-essential

./IsaacLab/isaaclab.sh --install
#or 
"./IsaacLab/isaaclab.sh -i"


# By default, the above will install all the learning frameworks. These include rl_games, rsl_rl, sb3, skrl, robomimic.
# Owever, if you only want to install a specific framework


./IsaacLab/isaaclab.sh --install rl_games  
# or 
"./IsaacLab/isaaclab.sh -i rl_games"

## Possible issue could result from a conflict between global and local installations from the above cmd.

In [ ]:
Installing collected packages: nvidia-nvtx-cu12, nvidia-nvjitlink-cu12, nvidia-curand-cu12, 
nvidia-cufile-cu12, nvidia-cuda-runtime-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-cupti-cu12, nvidia-cublas-cu12, nvidia-cusparse-cu12, nvidia-cufft-cu12, nvidia-cudnn-cu12, nvidia-cusolver-cu12, 
torch, torchvision Attempting uninstall: nvidia-nvtx-cu12 Found existing installation: nvidia-nvtx-cu12 12.6.77 Uninstalling nvidia-nvtx-cu12-12.6.77: 
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/usr/local/lib/python3.10/dist-packages/nvidia/__init__.py' Consider using the --user option or check the permissions. 
╭─    ~/RL/BeyondMimic   1 ✘  took 12s   isaacLab   at 13:34:15  ╰─

## Possible solution but not working 100%, then it is recommended to unset python path

In [ ]:
#Option 1
pip install --ignore-installed packaging -e .

# Option 2
conda install numpy packaging -y
pip install -e . --no-deps

# I created a .sh file with this code to run for automatically installtion within all folfder
# ~/R/B/IsaacLab/source 
# chmod +x install.sh

cd ~/RL/BeyondMimic/IsaacLab/source
for d in */ ; do
    if [ -f "$d/pyproject.toml" ] || [ -f "$d/setup.py" ]; then
        echo "Installing $d ..."
        pip install -e "$d" --no-deps
    else
        echo "Skipping $d (not a Python project)"
    fi
done


pip install flatdict

pip install --ignore-installed matplotlib==3.7.1

## Verifying the Isaac Lab installation

In [ ]:
# Option 1: Using the isaaclab.sh executable
# note: this works for both the bundled python and the virtual environment
./isaaclab.sh -p scripts/tutorials/00_sim/create_empty.py

# Option 2: Using python in the v env
python scripts/tutorials/00_sim/create_empty.py

## Another possible issue by runing the create_empty python script if you chosse to install manually the python package with the "install -e . --no-deps"

In [ ]:
 python IsaacLab/scripts/tutorials/00_sim/create_empty.py
Traceback (most recent call last):
  File "/home/jean/RL/BeyondMimic/IsaacLab/scripts/tutorials/00_sim/create_empty.py", line 20, in <module>
    from isaaclab.app import AppLauncher
ModuleNotFoundError: No module named 'isaaclab'

In [ ]:
python -m pip list | grep isaaclab

#isaaclab_rl                             0.4.2                /home/jean/Documents/IsaacLab/source/isaaclab_rl

# from the create_empty.py, I use to adapt the real name
#from isaaclab.app import AppLauncher
#from isaaclab_rl.app import AppLauncher

## Clone whole_body_tracking repository

In [ ]:
# Clone the repository (outside IsaacLab directory)
cd ~/RL/BeyondMimic
git clone https://github.com/HybridRobotics/whole_body_tracking.git

# Clone also this repository for motion conversion 
cd ~/RL/BeyondMimic
git clone https://bitbucket.org/theconstructcore/beyondmimic.git

# Clone also this lafan1_retargeting dataset motion 
cd ~/RL/BeyondMimic
git clone https://bitbucket.org/theconstructcore/lafan1_retargeting_dataset.git

## Installation

In [ ]:
# Enter the repository
cd whole_body_tracking

# Download robot description files from GCS
curl -L -o unitree_description.tar.gz https://storage.googleapis.com/qiayuanl_robot_descriptions/unitree_description.tar.gz && \
tar -xzf unitree_description.tar.gz -C source/whole_body_tracking/whole_body_tracking/assets/ && \
rm unitree_description.tar.gz

# Install using Isaac Lab's Python environment
python -m pip install -e source/whole_body_tracking

## Convert a .pkl to .csv (if needed)

In [ ]:
# Ensure to have joblib installed
pip install joblib

cd ~/RL/BeyondMimic/motions/TheConstruct
python3 convert_pbhc_to_g1_fixed.py

# Visualize .csv motion files

**LAFAN1's dataset provides a script to visualize the motion to be trained. To run it, just follow these steps:**

In [ ]:
#visualization script
https://huggingface.co/datasets/lvhaidong/LAFAN1_Retargeting_Dataset/blob/main/rerun_visualize.py

# Step 1: Set up a Conda virtual environment

#conda remove --name retarget --all
conda create -n retarget python=3.10 
conda activate retarget

# Step 2: Install dependencies  conda install conda=25.9.1
#unset PYTHONPATH
conda install pinocchio -c conda-forge
pip install numpy rerun-sdk==0.22.0 trimesh


#In case of issue "Solving environment" that take too long due to the "Anaconda" installation, try this:

#1) Set CONDA_SOLVER to libmamba, then run the installation process as follow:
#zsh --no-rcs
#unset PYTHONPATH 
#conda remove --name retarget --all
CONDA_SOLVER=libmamba conda create -n retarget -c conda-forge python=3.10 pinocchio numpy trimesh
conda activate retarget
unset PYTHONPATH
export PYTHONNOUSERSITE=1
pip install rerun-sdk==0.22.0
pip install joblib

#2) Or install micromamba whitouh conda 
cd ~
curl -L https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
./bin/micromamba shell init -s zsh -r ~/micromamba
exec zsh
ls ~/micromamba
micromamba --version

micromamba create -n robot-env -c conda-forge python=3.10 pinocchio numpy trimesh
micromamba activate robot-env
unset PYTHONPATH
export PYTHONNOUSERSITE=1   # to ignore ~/.local
pip install rerun-sdk==0.22.0
pip install joblib

python -c "import pinocchio as p; print('pinocchio:', p.__version__)"
python -c "import rerun as rr; print('rerun:', rr.__version__)"
python -c "import trimesh, numpy as np; print('trimesh:', trimesh.__version__, 'numpy:', np.__version__)"

# Step 3: Run the script
# run the script with parameters:
python rerun_visualize.py --file_name horse_stance_punch --robot_type g1

# Training Process

## Setting Up WandB Registry

### Create Registry Collection named "Motions" with artifact type "All Types"

In [ ]:
# Install wandb and login
#rm /home/jean/.local/bin/wandb
pip install wandb

# Or update
pip install --upgrade wandb

# login
wandb login

### Converting Motion Files

In [ ]:
# Convert retargeted CSV motions to NPZ format with maximum coordinates information:
# and upload to WandB registry

python scripts/csv_to_npz.py \
  --input_file /path/to/motion.csv \
  --input_fps 30 \
  --output_name motion_name \
  --headless

#export PYTHONPATH=$PWD:$PYTHONPATH
#export PYTHONPATH="$HOME/RL/BeyondMimic/whole_body_tracking:$PYTHONPATH"

python scripts/csv_to_npz.py --input_file ./motions/hooks_punch.csv --input_fps 30 \
  --output_name hooks_punch #--headless

### Verifying Motion Registry

In [ ]:
#Test that the WandB registry works by replaying the motion in Isaac Sim:
# Replay motion from registry
python scripts/replay_npz.py \
  --registry_name=your-organization-org/wandb-registry-motions/motion_name

  #In my case
python scripts/replay_npz.py --registry_name=jmayb-universidad-de-le-n-org/wandb-registry-motions/hooks_punch

#Troubleshooting:
    # Make sure WANDB_ENTITY is set to your organization name, not username

### Policy Training

**Once motions are uploaded to the WandB registry, the training policies can track them.**

In [ ]:
# Train policy for G1 robot on flat terrain
python scripts/rsl_rl/train.py --task=Tracking-Flat-G1-v0 --registry_name {your-username}-org/wandb-registry-motions/horse_stance_punch_with_standing --headless --logger wandb --log_project_name horse_stance_punch_with_standing --run_name run1_horse_stance_punch_with_standing --max_iterations 30000

#In my case 
python scripts/rsl_rl/train.py --task=Tracking-Flat-G1-v0 --registry_name jmayb-universidad-de-le-n-org/wandb-registry-motions/hooks_punch --logger wandb --log_project_name hooks_punch --run_name run1_hooks_punch --max_iterations 30000

### Possible issue

In [ ]:
# Potential Issue: ImportError and Omniverse USD warnings during startup
#
# During execution, users may encounter warnings from OmniUSD plugins such as:
#   - [omni.usd-abi.plugin] No setting found for 
#     '/rtx-defaults-transient/meshlights/forceDisable'
#   - [omni.graph.core.plugin] Could not find category 'Replicator:Annotators' for removal
# These warnings indicate missing optional configuration entries in the Omniverse 
# USD environment. They do not prevent simulation startup but may be confusing.
#
# Additionally, an ImportError may occur:
#   ImportError: cannot import name 'dump_pickle' from 'isaaclab.utils.io'
# This happens when the script relies on functions that have been removed or 
# relocated in newer versions of IsaacLab.
#


2025-11-17 14:03:01 [7,951ms] [Warning] [omni.usd-abi.plugin] No setting was found for '/rtx-defaults-transient/meshlights/forceDisable'
[9.772s] Simulation App Startup Complete
Traceback (most recent call last):
  File "/home/robotica/Documents/whole_body_tracking/scripts/rsl_rl/train.py", line 61, in <module>
    from isaaclab.utils.io import dump_pickle, dump_yaml
ImportError: cannot import name 'dump_pickle' from 'isaaclab.utils.io' (/home/robotica/IsaacLab/source/isaaclab/isaaclab/utils/io/__init__.py)
2025-11-17 14:03:03 [9,771ms] [Warning] [omni.graph.core.plugin] Could not find category 'Replicator:Annotators' for removal

In [ ]:
Error executing job with overrides: [] Traceback (most recent call last): File "/home/robotica/Documents/Exercice/IsaacLab/source/isaaclab_tasks/isaaclab_tasks/utils/hydra.py", line 101, in hydra_main func(env_cfg, agent_cfg, *args, **kwargs) 
File "/home/robotica/Documents/Exercice/whole_body_tracking/scripts/rsl_rl/train.py", line 155, in main runner.learn(num_learning_iterations=agent_cfg.max_iterations, init_at_random_ep_len=True) 
File "/home/robotica/miniconda3/envs/isaacLab/lib/python3.10/site-packages/rsl_rl/runners/on_policy_runner.py", line 159, in learn self.save(os.path.join(self.log_dir, f"model_{it}.pt")) 
File "/home/robotica/Documents/Exercice/whole_body_tracking/source/whole_body_tracking/whole_body_tracking/utils/my_on_policy_runner.py", 
line 38, in save self.env.unwrapped, self.alg.policy, normalizer=self.obs_normalizer, path=policy_path, filename=filename AttributeError: 'MotionOnPolicyRunner' object has no attribute 'obs_normalizer'

### Possible solution

In [ ]:
# Recommended Actions:
#   1. Ensure that your scripts are compatible with the installed IsaacLab version.
#   2. If necessary, use an IsaacLab version adapted for the current codebase.
#   3. Replace deprecated or removed functions (e.g., 'dump_pickle') with 
#      their updated equivalents or Python standard library alternatives.

In [ ]:
# Check whole_body_tracking/source/whole_body_tracking/whole_body_tracking/utils/my_on_policy_runner.py
save_policy(self.env.unwrapped, self.alg.policy, normalizer=self.obs_normalizer, ...)


# Try to get the obs normalizer from the algorithm (new RSL-RL API)
normalizer = getattr(self.alg, "obs_normalizer", None)

# Replace 
normalizer=self.obs_normalizer
# With
normalizer=normalizer,


In [ ]:
python scripts/rsl_rl/train.py --task=Tracking-Flat-G1-v0 --registry_name jmayb-universidad-de-le-n-org/wandb-registry-motions/hooks_punch --logger wandb --log_project_name hooks_punch --run_name run1_hooks_punch --max_iterations 30000

### Policy Evaluation

**After training, evaluate the policy in Isaac Sim to visualize performance.**

In [ ]:
# Play trained policy
python scripts/rsl_rl/play.py \
  --task=Tracking-Flat-G1-v0 \
  --num_envs=2 \
  --wandb_path=your_organization/project_name/run_id

# In my case
python scripts/rsl_rl/play.py \
--task=Tracking-Flat-G1-v0 \
--num_envs=2 \
--wandb_path=jmayb-universidad-de-le-n/hooks_punch/am8zy8tb


### Possible issues

In [ ]:
# Due to recent updates in IsaacLab, certain incompatibilities may arise, 
# including an AttributeError of the form:
#     'OnPolicyRunner' object has no attribute 'obs_normalizer'
#
# This occurs because the current version of RSL-RL no longer relies on an 
# external normalizer. Observation normalization is now handled internally 
# within the MLP layers, provided that the corresponding configuration 
# dictionaries include valid normalization parameters.
#
# In this configuration, both dictionaries — actor_obs_normalization: {} 
# and critic_obs_normalization: {} — are empty, implying that no observation 
# normalization is applied. This behavior is verified through the runner 
# output generated by:
#     print("RUNNER DICT:", ppo_runner.__dict__)

Error executing job with overrides: []
Traceback (most recent call last):
  File "/home/jean/RL/BeyondMimic/IsaacLab/source/isaaclab_tasks/isaaclab_tasks/utils/hydra.py", line 101, in hydra_main
    func(env_cfg, agent_cfg, *args, **kwargs)
  File "/home/jean/RL/BeyondMimic/whole_body_tracking/scripts/rsl_rl/play.py", line 155, in main
    normalizer=ppo_runner.obs_normalizer,
AttributeError: 'OnPolicyRunner' object has no attribute 'obs_normalizer'

Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.


# With the latest version of IsaacLab, the 'obs_normalizer' attribute is no 
# longer accessible through ppo_runner.obs_normalizer. Observation 
# normalization is now embedded directly within the Agent object itself. 
# Consequently, the code must be adapted by explicitly setting the normalizer 
# to None when no normalization is required.

    normalizer = getattr(ppo_runner.alg, "obs_rms", None)
    if normalizer is None and hasattr(ppo_runner.alg, "actor_critic"):
        normalizer = getattr(ppo_runner.alg.actor_critic, "obs_rms", None)

    export_motion_policy_as_onnx(
        env.unwrapped,
        ppo_runner.alg.policy,
        normalizer=normalizer,
        path=export_model_dir,
        filename="policy.onnx",
    )

In [ ]:
# This error originates from a mismatch between the action dimensions expected 
# by the IsaacLab ActionManager and the action tensor returned by the policy. 
# Specifically, the ActionManager verifies that:
#       self.total_action_dim == action.shape[1]
# and raises an IndexError when the action tensor lacks the second dimension 
# (e.g., when the policy outputs a 1D vector instead of a batched 2D tensor).
#
# This typically occurs when:
#   - the environment is wrapped without vectorization,
#   - the action space was modified but the policy architecture was not 
#     updated accordingly,
#   - or when loading a checkpoint trained with a different action dimension.
#
# To resolve the issue, ensure that:
#   (1) actions are always provided as a batched tensor of shape (N, dim),
#   (2) the policy and environment configurations define consistent action 
#       dimensions, and
#   (3) any pre-trained checkpoints match the current environment's action 
#       specification.
#
# Setting HYDRA_FULL_ERROR=1 can provide additional diagnostics for debugging.

Error executing job with overrides: []
Traceback (most recent call last):
  File "/home/jean/RL/BeyondMimic/IsaacLab/source/isaaclab_tasks/isaaclab_tasks/utils/hydra.py", line 101, in hydra_main
    func(env_cfg, agent_cfg, *args, **kwargs)
  File "/home/jean/RL/BeyondMimic/whole_body_tracking/scripts/rsl_rl/play.py", line 178, in main
    obs, _, _, _ = env.step(actions)
  File "/home/jean/RL/BeyondMimic/IsaacLab/source/isaaclab_rl/isaaclab_rl/rsl_rl/vecenv_wrapper.py", line 157, in step
    obs_dict, rew, terminated, truncated, extras = self.env.step(actions)
  File "/home/jean/anaconda3/envs/isaacLab/lib/python3.10/site-packages/gymnasium/wrappers/common.py", line 393, in step
    return super().step(action)
  File "/home/jean/anaconda3/envs/isaacLab/lib/python3.10/site-packages/gymnasium/core.py", line 327, in step
    return self.env.step(action)
  File "/home/jean/RL/BeyondMimic/IsaacLab/source/isaaclab/isaaclab/envs/manager_based_rl_env.py", line 174, in step
    self.action_manager.process_action(action.to(self.device))
  File "/home/jean/RL/BeyondMimic/IsaacLab/source/isaaclab/isaaclab/managers/action_manager.py", line 381, in process_action
    if self.total_action_dim != action.shape[1]:
IndexError: tuple index out of range

Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.


In [ ]:
# I adapted the code in play2.py provided
python scripts/rsl_rl/play2.py \
--task=Tracking-Flat-G1-v0 \
--num_envs=2 \
--wandb_path=jmayb-universidad-de-le-n/hooks_punch/am8zy8tb

# Deployment with motion_tracking_controller
## Sim2Sim: validate the policy in Mujoco

In [ ]:
# Check the ros version (if needed)
printenv | grep ROS_DISTRO
ros2 doctor --report
apt list --installed | grep ros-humble

### Install install unitree_sdk2

In [ ]:
# C++
# Dependencies installation
sudo apt update
#sudo apt install ros-humble-ros2-control ros-humble-ros2-controllers ros-humble-joint-state-publisher
sudo apt install libyaml-cpp-dev libspdlog-dev libboost-all-dev libglfw3-dev

In [ ]:
git clone https://github.com/unitreerobotics/unitree_sdk2.git
cd unitree_sdk2/
mkdir build
cd build
cmake .. -DCMAKE_INSTALL_PREFIX=/opt/unitree_robotics
sudo make install

### Install MuJoCo (Native Manual Setup)

In [ ]:
# Official documentation link for MuJoCo installation
https://github.com/unitreerobotics/unitree_mujoco

# Create the MuJoCo directory 
mkdir -p ~/.mujoco

# Download the latest MuJoCo release
https://github.com/google-deepmind/mujoco/releases

wget https://github.com/google-deepmind/mujoco/releases/download/3.3.7/mujoco-3.3.7-linux-x86_64.tar.gz

# Move the archive to ~/.mujoco
mv mujoco-3.3.7-linux-x86_64.tar.gz ~/.mujoco/
cd ~/.mujoco

# Extract the archive
tar -xzvf mujoco-3.3.7-linux-x86_64.tar.gz

# Create a symbolic link named mujoco
# Many tools (including unitree_mujoco) expect this location: ~/.mujoco/mujoco
ln -s ~/.mujoco/mujoco-3.3.7 ~/.mujoco/mujoco

# Add MuJoCo to the PATH
gedit ~/.zshrc

export MUJOCO_DIR="$HOME/.mujoco/mujoco-3.3.7"
export LD_LIBRARY_PATH="$MUJOCO_DIR/lib:$LD_LIBRARY_PATH"
export LD_LIBRARY_PATH="$LD_LIBRARY_PATH:$MUJOCO_HOME/lib"
export PATH="$PATH:$MUJOCO_HOME/bin"

# Or 
echo 'export MUJOCO_DIR="$HOME/.mujoco/mujoco-3.3.7"' >> ~/.zshrc
echo 'LD_LIBRARY_PATH="$MUJOCO_DIR/lib:$LD_LIBRARY_PATH"' >> ~/.zshrc
echo 'export LD_LIBRARY_PATH="$LD_LIBRARY_PATH:$MUJOCO_HOME/lib"' >> ~/.zshrc
echo 'export PATH="$PATH:$MUJOCO_HOME/bin"' >> ~/.zshrc

#Check
echo $MUJOCO_HOME
echo $LD_LIBRARY_PATH
which simulate

### Install the Unitree MuJoCo Simulator

#### Clone the repo 

In [ ]:
git clone https://github.com/unitreerobotics/unitree_mujoco.git
cd unitree_mujoco

#### Create symbolic link inside the simulator

In [ ]:
cd simulate
ln -s ~/.mujoco/mujoco mujoco

# If the symbolic link was not created during the native mujoco installation, then:
ln -s ~/.mujoco/mujoco-3.3.7 mujoco

# Compile unitree_mujoco
cd unitree_mujoco/simulate
mkdir build && cd build
cmake ..
make -j4

### Test the simulator

In [ ]:
./build/unitree_mujoco -r g1 -s scene_23dof.xml

#### The simulator rise an error 

In [ ]:
free(): invalid pointer
zsh: IOT instruction (core dumped)  ./unitree_mujoco -r g1 -s scene_23dof.xml

╭─    ~/unitree_mujoco/simulate/build  on   main 
╰─ 

# or a DDS error

In [ ]:
source ~/mujoco-rl/bin/activate
#export PYTHONPATH=$PYTHONPATH:/home/jean/RL/LeggGym/unitree_rl_gym
export LEGGM_PATH="$HOME/RL/LeggGym/unitree_rl_gym"
export PYTHONPATH="$LEGGM_PATH:$PYTHONPATH"
source /opt/ros/humble/setup.bash # or zsh
source ~/ros2rob_ws/install/setup.bash # or zsh


#### Policy from WandB

In [ ]:
pip install wandb onnxruntime numpy
export WANDB_API_KEY=ton_token

In [ ]:
# Syntax
ros2 launch motion_tracking_controller mujoco.launch.py \
  wandb_path:=your_organization/project_name/run_id

# In my case
ros2 launch motion_tracking_controller mujoco.launch.py \
  wandb_path:=jmayb-universidad-de-le-n/hooks_punch/am8zy8tb

#### Policy ONNX local

In [ ]:
#Syntax 
ros2 launch motion_tracking_controller mujoco.launch.py \
  policy_path:=~/path/to/policy.onnx
  
# In my case
ros2 launch motion_tracking_controller mujoco.launch.py \
  policy_path:=/home/jean/RL/policies/policy.onnx


### Run the Simulation

In [ ]:
# I use to create a .sh for quick and easy launch
touch src/motion_tracking_controller/run_mujoco_ros.sh
# Make it executable
chmod +x src/motion_tracking_controller/run_mujoco_ros.sh
# then I run
./src/motion_tracking_controller/run_mujoco_ros.sh wandb_path:=jmayb-universidad-de-le-n/hooks_punch/am8zy8tb

head -n 1 ~/ros2rob_ws/install/motion_tracking_controller/lib/motion_tracking_controller/policy_node

#!/usr/bin/python3

sed -i '1c #!/home/jean/mujoco-rl/bin/python3' \
  ~/ros2rob_ws/install/motion_tracking_controller/lib/motion_tracking_controller/policy_node


  head -n 1 ~/ros2rob_ws/install/motion_tracking_controller/lib/motion_tracking_controller/policy_node

#!/home/jean/mujoco-rl/bin/python3

# Then I tested 
./src/motion_tracking_controller/run_mujoco_ros.sh \
  wandb_path:=jmayb-universidad-de-le-n/hooks_punch/am8zy8tb
